In [3]:
import os
from tqdm import tqdm
import pandas as pd
import txt_to_prob as tp

# df_sub_ID, df_sub_ID_SOL 생성
print("Load subject's ID")
df_PSG = pd.read_excel('subject_info\\Assigned_class_Excluded_on_age_nan_dot.xlsx')

df_sub_ID = df_PSG.copy()['PSG study Number#']

df_sub_ID_SOL = df_PSG.copy()[['PSG study Number#', 'Sleep latency (min)']]
df_sub_ID_SOL.index = df_sub_ID_SOL['PSG study Number#']
df_sub_ID_SOL.drop(columns=['PSG study Number#'], inplace=True)
print("-- Total: {}".format(len(df_sub_ID)))

# txt 파일 관련 setting
path_txt = 'E:\\samsung_original\\whole_data_txt\\data4' # txt 파일이 저장되어 있는 경로
filenames_txt = pd.Series(os.listdir(path_txt)) # 모든 txt 파일의 filename이 담겨 있는 pd.Series
print('Number of txt files: {}'.format(filenames_txt.size))

# 이미 probabilistic hypnogram이 생성된 subject list
# path_prob_hyp = 'E:\\probabilistic_hypnogram'
path_save = 'E:\\samsung_original\\whole_data_txt\\data4_prob'
list_existing_hyp = os.listdir(path_save)

Load subject's ID
-- Total: 4946
Number of txt files: 1130


C:\Users\CYS\AppData\Local\Temp\ipykernel_6984\3713641121.py:8: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype=datetime64[ns])
  df_PSG = pd.read_excel('subject_info\\Assigned_class_Excluded_on_age_nan_dot.xlsx')


In [4]:
# QNAN = []
# QNAN_with_conversion = []
# list_excluded_1 = []
# list_excluded_2 = []
# list_excluded_3 = []
for sub_ID in tqdm(df_sub_ID, desc='Processing data4 ... '):      
      # exclusion
      #     1. A1과 C4가 모두 존재 하지 않는 경우 (sub_ID가 포함된 파일 이름을 읽은 list의 요소 개수 검사)
      #     2. txt 파일이 비어 있는 경우 (txt2np class에서 txt 파일이 비어 있는 경우, empty array를 반환하도록 설정)
      #     3. TIB-SOL < 330 인 경우 (np2raw class에서 array의 길이와 sub_ID에 따른 sampling rate를 이용하여 직접 TIB 계산)

      sub_ID = sub_ID.upper()
      if sub_ID in list_existing_hyp:
            # 이미 probabilistic hypnogram이 있는 경우, pass !
            # print('Already processed subject ... ')
            continue 
      
      # Read txt files and extract eeg data for all channels
      txt2np = tp.txt2np(path_txt, filenames_txt, sub_ID) # initialize txt2np class
      txt2np.txt_filename_with_sub_ID() # get filenames containing sub_ID

      if len(txt2np.temp_txt_subjects) < 2:
            # exclusion !
            # A1과 C4가 모두 존재 하지 않는 경우
            list_excluded_1.append(sub_ID)
            continue

      np_all_eeg = txt2np.read_txt() # read EEG data into np.array

      if np_all_eeg.shape[0] == 0:
            # exclusion !
            # txt 파일이 비어 있는 경우 pass
            list_excluded_2.append(sub_ID)
            continue
      QNAN.append(txt2np.QNAN_dic) # SOL을 고려하기 직전까지 배제되지 않은 subject에 대한 QNAN 정보

      # Create mne.raw instance 
      # - data: np_all_eeg
      # - raw.info['subject_info']['his_id']: subject ID
      # - preprocessing: re_referencing + cropping
      # - raw.ch_names: ['F3-A2', 'F4-A1', 'C3-A2', 'C4-A1', 'O1-A2', 'O2-A1']

      SOL = df_sub_ID_SOL.loc[sub_ID, 'Sleep latency (min)']
      np2raw = tp.np2raw(sub_ID, SOL)
      np2raw.np2raw(np_all_eeg)

      # exclusion !
      # len_np - SOL < 330 인 경우, pass
      if (np_all_eeg.shape[1]/np2raw.temp_sfreq/60 - SOL) < 330:
            list_excluded_3.append(sub_ID)
            continue

      np2raw.re_ref()
      raw_re_ref_cropped = np2raw.raw_cropping()
      
      QNAN_with_conversion.append(txt2np.QNAN_dic) # 실제 conversion까지 완료된 subjec들에 대한 QNAN 정보

      # Automatic Sleep Staging
      # - apply ensamble using all six eeg channels
      # - automatiaclly saved
      #   current directory
      #   -- hypnograms
      #   -- -- subject_1
      #   -- -- subject_2
      #   -- -- ...
      #   -- -- subject_n
      #   -- -- -- predicted_hypnogram.csv
      #   -- -- -- probabilistic_hypnogram.csv
      automatic_staging = tp.automatic_staging()
      automatic_staging.get_hypnos_and_probs(raw_re_ref_cropped.copy())
      automatic_staging.ensamble_stagig(path_save)

Processing data4 ... : 100%|██████████| 4946/4946 [1:25:35<00:00,  1.04s/it]   


In [5]:
print(len(QNAN))
print(len(QNAN_with_conversion))

520
502


In [6]:
df_QNAN = pd.DataFrame(columns = ['A1', 'C4'])
for temp_dic in QNAN:
    if len(temp_dic.keys()) != 0:
        temp_sub_ID = list(temp_dic.keys())[0]
        df_QNAN.loc[temp_sub_ID] = [0, 0]
        if list(temp_dic.values())[0] == 'A1':
            df_QNAN.loc[temp_sub_ID, 'A1'] = 1
        elif list(temp_dic.values())[0] == 'C4':
            df_QNAN.loc[temp_sub_ID, 'C4'] = 1

df_QNAN_conversion = pd.DataFrame(columns = ['A1', 'C4'])
for temp_dic in QNAN_with_conversion:
    if len(temp_dic.keys()) != 0:
        temp_sub_ID = list(temp_dic.keys())[0]
        df_QNAN_conversion.loc[temp_sub_ID] = [0, 0]
        if list(temp_dic.values())[0] == 'A1':
            df_QNAN_conversion.loc[temp_sub_ID, 'A1'] = 1
        elif list(temp_dic.values())[0] == 'C4':
            df_QNAN_conversion.loc[temp_sub_ID, 'C4'] = 1


In [7]:
print("Number of QNAN in A1 \n{}".format(df_QNAN['A1'].value_counts()))
print("\nNumber of QNAN in C4 \n{}".format(df_QNAN['C4'].value_counts()))
df_QNAN

Number of QNAN in A1 
1    272
Name: A1, dtype: int64

Number of QNAN in C4 
0    272
Name: C4, dtype: int64


,A1,C4
PE150707,1,0
PE150708,1,0
PE150710,1,0
PE150727,1,0
PE150728,1,0
...,...,...
PE170942,1,0
PE170943,1,0
PE170947,1,0
PE170949,1,0


In [8]:
print("Number of QNAN in A1 \n{}".format(df_QNAN_conversion['A1'].value_counts()))
print("\nNumber of QNAN in C4 \n{}".format(df_QNAN_conversion['C4'].value_counts()))
df_QNAN_conversion

Number of QNAN in A1 
1    263
Name: A1, dtype: int64

Number of QNAN in C4 
0    263
Name: C4, dtype: int64


,A1,C4
PE150707,1,0
PE150708,1,0
PE150710,1,0
PE150727,1,0
PE150728,1,0
...,...,...
PE170942,1,0
PE170943,1,0
PE170947,1,0
PE170949,1,0


In [9]:
df_QNAN.to_csv('QNaN_inspection\\data4_QNaN.csv')
df_QNAN_conversion.to_csv('QNaN_inspection\\data4_QNaN_conversion.csv')

In [10]:
pd.Series(list_excluded_1).to_csv('excluded\\data4_excluded_1.csv')
pd.Series(list_excluded_2).to_csv('excluded\\data4_excluded_2.csv')
pd.Series(list_excluded_3).to_csv('excluded\\data4_excluded_3.csv')

C:\Users\CYS\AppData\Local\Temp\ipykernel_6984\731642772.py:2: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  pd.Series(list_excluded_2).to_csv('excluded\\data4_excluded_2.csv')
